In [ ]:
import numpy as np
import random

# Grid size and global parameters
GRID_SIZE = 12
GOAL = (11, 6)
START = (0, 6)
ACTIONS = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, Down, Left, Right

# Stochastic transition probabilities
INTENDED_PROB = 0.8
PERP_PROB = 0.1

# Maze grid (0 = free space, 1 = obstacle)
maze_grid = np.array([[1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1],
                      [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
                      [1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],
                      [1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1],
                      [1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1],
                      [1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1],
                      [1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],
                      [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],
                      [1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1],
                      [1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1],
                      [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
                      [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1]])

# Reward function based on the current and next state
def get_reward(state, next_state):
    if next_state == GOAL:
        return 100
    elif maze_grid[next_state[0]][next_state[1]] == 1:  # Hitting an obstacle
        return -10
    else:
        return -1

# Check if a state is valid (within bounds and not an obstacle)
def is_valid_state(state):
    return 0 <= state[0] < GRID_SIZE and 0 <= state[1] < GRID_SIZE and maze_grid[state[0]][state[1]] == 0

# Get the next state based on the chosen action and stochastic transitions
def get_next_state(state, action):
    next_state = (state[0] + action[0], state[1] + action[1])
    if is_valid_state(next_state):
        return next_state
    else:
        return state  # Stay in the same state if the move is invalid

# Get the list of possible next states with probabilities
def get_possible_transitions(state, action):
    transitions = []
    intended_next = get_next_state(state, action)
    transitions.append((INTENDED_PROB, intended_next))
    
    # Perpendicular moves
    if action == (-1, 0) or action == (1, 0):  # Moving Up or Down
        transitions.append((PERP_PROB, get_next_state(state, (0, -1))))  # Left
        transitions.append((PERP_PROB, get_next_state(state, (0, 1))))   # Right
    else:  # Moving Left or Right
        transitions.append((PERP_PROB, get_next_state(state, (-1, 0))))  # Up
        transitions.append((PERP_PROB, get_next_state(state, (1, 0))))   # Down
    
    return transitions

# Policy Iteration parameters
GAMMA = 0.9
THETA = 0.01
EPISODES = 1000

# Initialize random policy and value function
policy = np.random.choice(len(ACTIONS), (GRID_SIZE, GRID_SIZE))  # Random policy
value_function = np.zeros((GRID_SIZE, GRID_SIZE))  # Value function
q_function = np.zeros((GRID_SIZE, GRID_SIZE, len(ACTIONS)))  # Q-function

# Policy iteration with G_t calculation
def policy_iteration():
    global policy, value_function, q_function
    returns = []  # To store cumulative return G_t for each episode

    is_policy_stable = False
    while not is_policy_stable:
        # Policy Evaluation
        while True:
            delta = 0
            for i in range(GRID_SIZE):
                for j in range(GRID_SIZE):
                    state = (i, j)
                    if state == GOAL:
                        continue  # Skip goal state
                    
                    v = value_function[i][j]
                    action = ACTIONS[policy[i][j]]
                    value_sum = 0
                    
                    # Calculate expected value considering stochastic transitions
                    for prob, next_state in get_possible_transitions(state, action):
                        reward = get_reward(state, next_state)
                        value_sum += prob * (reward + GAMMA * value_function[next_state[0]][next_state[1]])
                    
                    value_function[i][j] = value_sum
                    delta = max(delta, abs(v - value_function[i][j]))
            
            if delta < THETA:
                break

        # Calculate G_t for the current episode
        G_t = 0
        state = START
        episode_rewards = []
        
        while state != GOAL:
            action = ACTIONS[policy[state[0]][state[1]]]
            next_state = get_next_state(state, action)
            reward = get_reward(state, next_state)
            episode_rewards.append(reward)
            state = next_state

        # Sum the discounted rewards to compute G_t
        for k, reward in enumerate(episode_rewards):
            G_t += (GAMMA ** k) * reward

        returns.append(G_t)

        # Policy Improvement
        is_policy_stable = True
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                state = (i, j)
                if state == GOAL:
                    continue  # Skip goal state
                
                old_action = policy[i][j]
                
                # Find the action that maximizes expected value with stochastic transitions
                action_values = []
                for action_idx, action in enumerate(ACTIONS):
                    value_sum = 0
                    for prob, next_state in get_possible_transitions(state, action):
                        reward = get_reward(state, next_state)
                        value_sum += prob * (reward + GAMMA * value_function[next_state[0]][next_state[1]])
                    action_values.append(value_sum)
                    
                    # Update q_function for q(s, a)
                    q_function[i][j][action_idx] = value_sum
                
                best_action = np.argmax(action_values)
                policy[i][j] = best_action

                if old_action != best_action:
                    is_policy_stable = False

    return policy, value_function, q_function, returns

# Run policy iteration
optimal_policy, optimal_value_function, optimal_q_function, cumulative_returns = policy_iteration()

# Print the results
print("Optimal Policy:")
print(optimal_policy)

print("\nOptimal Value Function (V*):")
print(optimal_value_function)

print("\nOptimal Q Function (Q*(s, a)):")
print(optimal_q_function)

print("\nCumulative Returns (G_t) per Episode:")
print(cumulative_returns)
